# Dimensionality reduction with cisTopic

Using inline R magic

In [1]:
import numpy as np
import scanpy as sc 
import pandas as pd

### r2py set-up

In [2]:
import rpy2.rinterface_lib.callbacks
import logging

In [3]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

I use `anndata2ri` to handle conversion of the sparse matrix (not too straightforward with `rpy2` alone)

In [4]:
import anndata2ri
anndata2ri.activate()

Loading the `rpy2` extension enables cell magic to be used. This runs R code in jupyter notebook cells.

In [5]:
%load_ext rpy2.ipython

In [ ]:
%%R
## Install dependencies
devtools::install_github("aertslab/RcisTarget")
devtools::install_github("aertslab/AUCell")
        
devtools::install_github("aertslab/cisTopic")
library(cisTopic)

These packages have more recent versions available.
It is recommended to update all of them.
Which would you like to update?

 1: All                                       
 2: CRAN packages only                        
 3: None                                      
 4: matrixStats (0.56.0    -> 0.57.0  ) [CRAN]
 5: R.methodsS3 (1.8.0     -> 1.8.1   ) [CRAN]
 6: glue        (1.4.1     -> 1.4.2   ) [CRAN]
 7: digest      (0.6.25    -> 0.6.27  ) [CRAN]
 8: vctrs       (0.3.2     -> 0.3.5   ) [CRAN]
 9: rlang       (0.4.7     -> 0.4.8   ) [CRAN]
10: bit64       (4.0.2     -> 4.0.5   ) [CRAN]
11: RSQLite     (2.2.0     -> 2.2.1   ) [CRAN]
12: XML         (3.98-1.20 -> 3.99-0.3) [CRAN]
13: pillar      (1.4.6     -> 1.4.7   ) [CRAN]
14: magrittr    (1.5       -> 2.0.1   ) [CRAN]
15: cli         (2.0.2     -> 2.2.0   ) [CRAN]
16: withr       (2.2.0     -> 2.3.0   ) [CRAN]
17: R6          (2.4.1     -> 2.5.0   ) [CRAN]
18: jsonlite    (1.7.0     -> 1.7.1   ) [CRAN]
19: R.oo        (1.23.0    -

Enter one or more numbers, or an empty line to skip updates: 


✔  checking for file ‘/tmp/RtmpS6oAq3/remotes4af49de9c87/aertslab-RcisTarget-fd09a60/DESCRIPTION’ (1.8s)
─  preparing ‘RcisTarget’:
✔  checking DESCRIPTION meta-information ...
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  looking to see if a ‘data/datalist’ file should be added
─  building ‘RcisTarget_1.9.2.tar.gz’ (2.6s)
   
These packages have more recent versions available.
It is recommended to update all of them.
Which would you like to update?

 1: All                                       
 2: CRAN packages only                        
 3: None                                      
 4: matrixStats (0.56.0    -> 0.57.0  ) [CRAN]
 5: magrittr    (1.5       -> 2.0.1   ) [CRAN]
 6: rlang       (0.4.7     -> 0.4.8   ) [CRAN]
 7: R6          (2.4.1     -> 2.5.0   ) [CRAN]
 8: digest      (0.6.25    -> 0.6.27  ) [CRAN]
 9: R.methodsS3 (1.8.0     -> 1.8.1   ) [CRAN]
10: glue        (1.4.1     -> 1.4.2   ) [CRA

Enter one or more numbers, or an empty line to skip updates: 


✔  checking for file ‘/tmp/RtmpS6oAq3/remotes4af4b0f3242/aertslab-AUCell-be87c73/DESCRIPTION’ (1.3s)
─  preparing ‘AUCell’:
✔  checking DESCRIPTION meta-information ...
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘AUCell_1.13.0.tar.gz’
   
These packages have more recent versions available.
It is recommended to update all of them.
Which would you like to update?

 1: All                                            
 2: CRAN packages only                             
 3: None                                           
 4: matrixStats  (0.56.0      -> 0.57.0    ) [CRAN]
 5: glue         (1.4.1       -> 1.4.2     ) [CRAN]
 6: digest       (0.6.25      -> 0.6.27    ) [CRAN]
 7: vctrs        (0.3.2       -> 0.3.5     ) [CRAN]
 8: rlang        (0.4.7       -> 0.4.8     ) [CRAN]
 9: colorspace   (1.4-1       -> 2.0-0     ) [CRAN]
10: R6           (2.4.1       -> 2.5.0     ) [CRAN]
11: labeling     (0.3    

Enter one or more numbers, or an empty line to skip updates: 


✔  checking for file ‘/tmp/RtmpS6oAq3/remotes4af1e271fd6/aertslab-cisTopic-4ae8599/DESCRIPTION’ (1.3s)
─  preparing ‘cisTopic’:
✔  checking DESCRIPTION meta-information ...
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  looking to see if a ‘data/datalist’ file should be added
─  building ‘cisTopic_0.3.0.tar.gz’ (11s)


### Load data

In [6]:
outdir = "/home/jovyan/data/lung_adult_scATAC/"
experiment_prefix = 'lungAdult_'

In [7]:
adata = sc.read_h5ad(outdir + experiment_prefix + '_ATAC_raw.h5ad')

### Set-up for cisTopic

Convert `anndata` object to `SingleCellExperiment` (conversion takes a while)

In [9]:
adata.X = adata.layers["binary_raw"]

In [10]:
%%R -i adata
adata

class: SingleCellExperiment 
dim: 96161 57992 
metadata(0):
assays(2): X binary_raw
rownames(96161): chr1:816942-817444 chr1:826600-827805 ...
  chrY:21574860-21575895 chrY:22366897-22367799
rowData names(15): peak_width exon ... total_counts log1p_total_counts
colnames(57992): 59-AAACGAAAGATATGAC-1 59-AAACGAAAGGTAGGAA-1 ...
  04-TTTGTGTTCGGATAGG-1 04-TTTGTGTTCTCGTAGA-1
colData names(1): cellatac_clusters
reducedDimNames(0):
spikeNames(0):
altExpNames(0):


In [11]:
%%R
library(cistopic)
library(Matrix)

assays(adata, "X")


Error in library(cistopic) : there is no package called ‘cistopic’


In [ ]:
# Initialize cisTopic object 
cisTopicObject <- createcisTopicObject(adata, project.name='ATAC_cisTopic')